In [1]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense,Activation,Attention,Bidirectional,Concatenate
from tensorflow.keras.models import load_model

In [2]:
##### Limit GPU for training ###
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
from pythainlp import thai_letters

In [4]:
label_map_0 = ["0","1","2","3","4","5","6","7","8","9","-",".","/","(",")","\"","&"]
label_map_1 = list(thai_letters)

In [5]:
char_set = label_map_0+label_map_1

In [6]:
char2int = { char_set[x]:x for x in range(len(char_set)) }
int2char = { char2int[x]:x for x in char_set }

In [7]:
count = len(char_set)
codes = ['\t' ,'\n', ' ']

In [8]:
for i in range(len(codes)):
    code = codes[i]
    char2int[code]=count
    int2char[count]=code
    count+=1

In [9]:
def decode_sequence(input_seq,num_dec_tokens=len(int2char),max_dec_len=75):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_dec_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, char2int['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    decoded_list =[]
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int2char[sampled_token_index]
        decoded_sentence += sampled_char
        decoded_list.append(sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_dec_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_dec_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence,decoded_list

In [23]:
encoder_model = load_model('encoder_v10.h5',compile=False)
decoder_model = load_model('decoder_v10.h5',compile=False)

In [24]:
def prep_input_text(text,max_len=78):
    text_len = len(text) + text.count(' ')
    encoder_input_data = np.zeros( (1 , max_len , len(char2int)),dtype='float32' )
    for i,char in enumerate(list(text)):
        encoder_input_data[ 0, i , char2int[char] ] = 1
    return encoder_input_data

In [25]:
def generate_correct_text(text):
    encoder_input_data = prep_input_text(text)
    txt,lst = decode_sequence(encoder_input_data)
    text_result = str(''.join(lst[:-1]))
    return text_result

In [26]:
text = str('โรงพยาบาลเกษมรา')

In [27]:
generate_correct_text(text)

'โรงพยาบาลเกษมราษฎร์'

In [37]:
txt =''.join(text.split())

In [28]:
df_hos =pd.read_csv('../datasets/hospital_augment_test6_3.csv')

In [29]:
hos_name  =list(df_hos['gen_data'])

In [30]:
hos_text =[]
for name in hos_name[:100]:
    new_name = str(name.replace('\u200b',''))
    hos_text.append(generate_correct_text(new_name))
    print(name.replace('\u200b',''), '>', generate_correct_text(new_name))

คลินิกกรุงเ3ทพพัทยาสาขาจอมเทียน > คลินิกกรุงเทพพัทยาสาขาจอมเทียน
คลินิชกกุงเทพระยองส"าขาบ่อวิน > คลินิกกรุงเทพระยองสาขาบ่อวิน
คลินิกชรุงเทพระยองสาขาปลวกแดง > คลินิกกรุงเทพระยองสาขาปลวกแดง
คลินิกไท์ยอินเตอร์การแพ&ยท(เกาะพงัน) > คลินิกไทยอินเตอร์การแพทย์(เกาะพงัน)
คลินิกรักสโขภาพจ.ระยอง > คลินิกรักสุขภาพจ.ระยอง
คลินิกโรงพยาบาลกรุงเทพพัทยฮาสาขานาเ"ลโือ > คลินิกโรงพยาบาลกรุงเทพพัทยาสาขานาเกลือ
คลินิกโรงทยาบาลกรุงเทพพัทยาสาขาบุญสัมพันธ์ > คลินิกโรงพยาบาลกรุงเทพพัทยาสาขาบุญสัมพันธ์
คลินิกโงพยาบาลกร6ุงเทพสม"ุย > คลินิกโรงพยาบาลกรุงเทพสมุย
คลินิกโรงพฒญาบาลสมิติเวชศรีราชา(อีส3เทิร์นซีบอร์ด) > คลินิกโรงพยาบาลสมิติเวชศรีราชา(อีสเทิร์นซีบอร์ด)
คลินิกเวชกรรมกสรุงไเทพมวกเหลก > คลินิกเวชกรรมกรุงเทพมวกเหล็ก
คลินณิกเวชกรร่มกล้วยน้ำไทสาขาชุมชน70ไร่ > คลินิกเวชกรรมกล้วยน้ำไทสาขาชุมชน70ไร่
คลินิกวชกร2ฅรมกล้วยน้ำไทสากขาท่ฤุงสองห้อง > คลินิกเวชกรรมกล้วยน้ำไทสาขาทุ่งสองห้อง
คลินิกเวชกรรมกล้วยน้ำไชทสาขาสุขุมวิท5ึ6 > คลินิกเวชกรรมกล้วยน้ำไทสาขาสุขุมวิท56
คลินิกเวชกฉรรมกล้วยน้ำไทสขาสุขุมดิท93 > คลินิกเวชกรรมกล้

In [84]:
count  = 0
for i,val in enumerate(hos_name):
    if val == hos_text[i]:
        count +=1
    else:
        print(hos_text[i],'>',val)

In [67]:
len(hos_name),len(hos_text)

(1375, 1375)